In [ ]:
#| default_exp prune.pruner

In [ ]:
#| export
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_pruning as tp
from torch_pruning.pruner import function

import numpy as np
import torch
import torch.nn as nn
import pickle
from itertools import cycle
from fastcore.basics import store_attr, listify, true
from fasterai.core.criteria import *
from fastai.vision.all import *

In [ ]:
#| include: false
from nbdev.showdoc import *

In [ ]:
#| export
class Pruner():
    def __init__(self, model, context, criteria, layer_type=[nn.Conv2d, nn.Linear, nn.LSTM], example_inputs=torch.randn(1,3,224,224), ignored_layers=None):
        store_attr()
        self.ignored_layers = []
        self.ignored_params = []
        if ignored_layers is not None:
            for layer in ignored_layers:
                if isinstance(layer, nn.Module):
                    self.ignored_layers.extend(list(layer.modules()))
                elif isinstance(layer, nn.Parameter):
                    self.ignored_params.append(layer)

        self.DG = tp.DependencyGraph()
        self.DG.build_dependency(self.model, example_inputs=example_inputs.to(next(model.parameters()).device), ignored_params=self.ignored_params)
        self._save_init_state()
        self._reset_threshold()
        self.init_num_groups = None

    def compute_threshold(self, sparsity):
        self.global_importance = {}
        for ix, grp in enumerate(self.DG.get_all_groups(root_module_types=self.layer_type, ignored_layers=self.ignored_layers)):
            imp = self.group_importance(grp)
            self.global_importance[ix] = imp

        global_imp = torch.cat(list(self.global_importance.values()), dim=0)

        self.init_num_groups = self.init_num_groups or len(global_imp)
        n_pruned = np.clip(int((1-sparsity/100)*self.init_num_groups), 1, len(global_imp))
        self.global_threshold = torch.topk(global_imp, n_pruned)[0].min()

    def prune_group(self, group, ix, sparsity, round_to):
        module = group[0][0].target.module
        pruning_fn = group[0][0].handler
        pruning_idxs = self.prune_method(group, ix, sparsity, round_to)
        
        group = self.DG.get_pruning_group(module, pruning_fn, pruning_idxs.tolist())
        group.prune()
    
    def prune_model(self, sparsity, round_to=None):
        if self.context=='global': self.compute_threshold(sparsity)
        for ix, group in enumerate(self.DG.get_all_groups(root_module_types=self.layer_type, ignored_layers=self.ignored_layers)):
            self.prune_group(group, ix, sparsity, round_to)

    def prune_method(self, group, ix, sparsity, round_to):
        if self.context=='global':
            imp = self.global_importance[ix]
            n_pruned = max(1, int(imp.ge(self.global_threshold).sum()))
        else:
            imp = self.group_importance(group)
            
            if self.DG.is_out_channel_pruning_fn(group[0].dep.handler):
                prunable_channels = group[0].dep.target.module._init_out_channels
            else:
                prunable_channels = group[0].dep.target.module._init_in_channels

            n_pruned = max(1, int((1-sparsity/100)*prunable_channels))
 
        threshold = torch.topk(imp, int(self._rounded_sparsity(torch.tensor(n_pruned), round_to)))[0].min() if round_to else torch.topk(imp, n_pruned)[0].min()

        return imp.lt(threshold).nonzero().view(-1)
    
                
    def updated_sparsity(self, m, sparsity):
        init_channels = m._init_out_channels
        return sparsity
                
    def _save_init_state(self):
        for m in self.model.modules():
            if hasattr(m, 'weight'):
                setattr(m, '_init_out_channels', self.DG.get_out_channels(m))
                setattr(m, '_init_in_channels', self.DG.get_in_channels(m))

    def _rounded_sparsity(self, n_to_prune, round_to):
        return max(round_to*torch.floor(n_to_prune/round_to), round_to)
    
    def _reset_threshold(self):
        self.global_threshold=None
    
    def group_importance(self, group):
        handler_map = {
            function.prune_conv_out_channels: 'filter',
            #function.prune_linear_out_channels: 'row',
            #function.prune_linear_in_channels: 'column',
            function.prune_conv_in_channels: 'shared_kernel',
            # Additional handlers can be added here
        }

        group_imp = []
        group_idxs = []

        for i, (dep, idxs) in enumerate(group):
            if dep.handler in handler_map:
                impo = self.criteria(dep.target.module, handler_map.get(dep.handler), squeeze=True)
                group_imp.append(impo)
                group_idxs.append(group[i].root_idxs)

        reduced_imp = torch.zeros_like(group_imp[0])

        for i, (imp, root_idxs) in enumerate(zip(group_imp, group_idxs)):
            imp = imp.to('cpu')
            reduced_imp = reduced_imp.to('cpu')
            reduced_imp.scatter_add_(0, torch.tensor(root_idxs, device=imp.device), imp)

        reduced_imp /= len(group_imp)

        return reduced_imp.to(default_device())

In [ ]:
show_doc(Pruner.prune_model)

---

[source](https://github.com/nathanhubens/fasterai/tree/master/blob/master/fasterai/prune/pruner.py#L51){target="_blank" style="float:right; font-size:smaller"}

### Pruner.prune_model

>      Pruner.prune_model (sparsity, round_to=None)

Let's try the `Pruner` with a VGG16 model

In [ ]:
torch.topk(torch.randn(1000), int(max(8*torch.floor(torch.tensor(30)/8), 8)))[0].min()

tensor(2.0126)

In [ ]:
torch.topk(torch.randn(1000), 30)[0].min()

tensor(2.0065)

In [ ]:
max(8*torch.floor(torch.tensor(30)/8), 8)

tensor(24.)

In [ ]:
model = resnet18().to('cuda:0')
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

The `Pruner`can either remove filters based on `local` criteria (i.e. each layer will be trimmed of the same % of filters)

In [ ]:
pruner = Pruner(model, 'local', large_final, layer_type=[nn.Conv2d])
pruner.prune_model(3, round_to=8)
print(model)

pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224).to('cuda:0')); pruned_macs, pruned_params

ResNet(
  (conv1): Conv2d(3, 48, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

(1393209728.0, 10405416)

The `Pruner`can also remove filters based on `global` criteria (i.e. each layer will be trimmed of a different % of filters, but we specify the sparsity of the whole network)

In [ ]:
model = vgg16_bn()

In [ ]:
pruner = Pruner(model, 'global', large_final, layer_type=[nn.Conv2d])
pruner.prune_model(50)
print(model)

pruned_macs, pruned_params = tp.utils.count_ops_and_params(model, torch.randn(1,3,224,224)); pruned_macs, pruned_params

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

(9945391172.0, 126347419)